<a href="https://colab.research.google.com/github/diaboloshogunate/GoogleColabML/blob/main/fashionMNIST-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import sys
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor

In [ ]:
# Enable GPU if avialable
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, stack):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = stack

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# Define train/test
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# parameters
momentum = 0
loss_fn = nn.CrossEntropyLoss()
batch_size = 30
learning_rate = 0.001
epochs = 2
layers = [nn.Linear(28*28, 1024), nn.ReLU(), nn.Linear(1024, 10)]

In [ ]:
# datasets
trainset = torchvision.datasets.FashionMNIST('./data', download=True, train=True, transform=ToTensor())
testset = torchvision.datasets.FashionMNIST('./data', download=True, train=False, transform=ToTensor())

In [ ]:
# data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# model
model = NeuralNetwork(nn.Sequential(*layers)).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.320202  [    0/60000]
loss: 2.220774  [ 3000/60000]
loss: 2.170064  [ 6000/60000]
loss: 2.059722  [ 9000/60000]
loss: 1.922381  [12000/60000]
loss: 1.970676  [15000/60000]
loss: 1.903179  [18000/60000]
loss: 1.936288  [21000/60000]
loss: 1.695264  [24000/60000]
loss: 1.727793  [27000/60000]
loss: 1.596017  [30000/60000]
loss: 1.512745  [33000/60000]
loss: 1.671964  [36000/60000]
loss: 1.540066  [39000/60000]
loss: 1.522604  [42000/60000]
loss: 1.394728  [45000/60000]
loss: 1.432420  [48000/60000]
loss: 1.359246  [51000/60000]
loss: 1.091484  [54000/60000]
loss: 1.138226  [57000/60000]
Test Error: 
 Accuracy: 65.2%, Avg loss: 1.237380 

Epoch 2
-------------------------------
loss: 1.146760  [    0/60000]
loss: 1.179499  [ 3000/60000]
loss: 1.146505  [ 6000/60000]
loss: 1.182852  [ 9000/60000]
loss: 1.177042  [12000/60000]
loss: 1.028485  [15000/60000]
loss: 1.147798  [18000/60000]
loss: 0.985244  [21000/60000]
loss: 1.302856  [24000/60000